In [ ]:
# Mandi Analytics & Advisory
# Load CSVs, clean, compute KPIs, train simple model, export insights

import pandas as pd
import numpy as np
from pathlib import Path

base = Path('E:/farmarMAin')
paths = [
    base / 'Dataset.csv',
    base / 'Agri Market Dataset.csv',
    base / 'Price_Agriculture_commodities_Week.csv'
]

frames = []
for p in paths:
    if p.exists():
        try:
            df = pd.read_csv(p)
            df['__source'] = p.name
            frames.append(df)
        except Exception as e:
            print('Failed reading', p, e)
    else:
        print('Missing', p)

len(frames), [f.shape for f in frames]


In [ ]:
# Basic normalization helpers

def to_title(x):
    if pd.isna(x):
        return x
    s = str(x).strip().lower()
    return ' '.join(w.capitalize() for w in s.split())

norms = []
for df in frames:
    cols = {c.lower().strip(): c for c in df.columns}
    # Try map common fields
    state = cols.get('state') or cols.get('state_name')
    district = cols.get('district')
    commodity = cols.get('commodity') or cols.get('crop') or cols.get('product')
    market = cols.get('market') or cols.get('market_name')
    modal = cols.get('modal_price') or cols.get('price') or cols.get('modal')
    minp = cols.get('min_price')
    maxp = cols.get('max_price')
    arrival = cols.get('arrivals_in_qtl') or cols.get('arrival') or cols.get('arrivals')
    date = cols.get('arrival_date') or cols.get('date') or cols.get('week')

    nd = pd.DataFrame()
    if commodity in df: nd['commodity'] = df[commodity].map(to_title)
    if market in df: nd['market'] = df[market].map(to_title)
    if state in df: nd['state'] = df[state].map(to_title)
    if district in df: nd['district'] = df[district].map(to_title)
    if modal in df: nd['modal_price'] = pd.to_numeric(df[modal], errors='coerce')
    if minp in df: nd['min_price'] = pd.to_numeric(df[minp], errors='coerce')
    if maxp in df: nd['max_price'] = pd.to_numeric(df[maxp], errors='coerce')
    if arrival in df: nd['arrival_qtl'] = pd.to_numeric(df[arrival], errors='coerce')
    if date in df: nd['date'] = pd.to_datetime(df[date], errors='coerce')
    nd['__source'] = df['__source'].iloc[0]
    norms.append(nd)

combined = pd.concat(norms, ignore_index=True)
combined = combined.dropna(subset=['commodity'])
combined.head(3), combined.shape


In [ ]:
# KPIs by commodity/state

kpi = combined.copy()
kpi['month'] = kpi['date'].dt.to_period('M')

agg = kpi.groupby(['commodity','state','month']).agg(
    avg_modal=('modal_price','mean'),
    min_modal=('modal_price','min'),
    max_modal=('modal_price','max'),
    total_arrival=('arrival_qtl','sum'),
    markets=('market','nunique')
).reset_index()

# Momentum: last 3 months average change
last3 = (agg.sort_values('month')
           .groupby(['commodity','state'])
           .tail(3)
           .groupby(['commodity','state'])
           .agg(momentum=('avg_modal', lambda s: (s.iloc[-1]-s.iloc[0]) / (abs(s.iloc[0])+1e-6)))
           .reset_index())

agg = agg.merge(last3, on=['commodity','state'], how='left')
agg.head(3)


In [ ]:
# Simple advisory rules

advice = []
for (commodity, state), grp in agg.groupby(['commodity','state']):
    g = grp.sort_values('month').tail(3)
    if g.empty:
        continue
    last_avg = g['avg_modal'].iloc[-1]
    mom = g['avg_modal'].iloc[-1] - g['avg_modal'].iloc[0]
    momentum = (mom / (abs(g['avg_modal'].iloc[0]) + 1e-6))
    arrival = g['total_arrival'].sum()

    status = 'hold'
    rationale = []
    if momentum > 0.05:
        status = 'bullish'
        rationale.append('Price rising in last 3 months')
    elif momentum < -0.05:
        status = 'bearish'
        rationale.append('Price falling in last 3 months')
    else:
        rationale.append('Stable prices')

    if arrival is not None and arrival < g['total_arrival'].mean() * 2:
        rationale.append('Supply normal/low')
    else:
        rationale.append('High arrivals (possible pressure)')

    advice.append({
        'commodity': commodity,
        'state': state,
        'last_avg_price': float(last_avg) if pd.notna(last_avg) else None,
        'momentum': float(momentum) if pd.notna(momentum) else None,
        'status': status,
        'rationale': rationale
    })

len(advice)


In [ ]:
# Export insights JSON for frontend/backend use

import json
out_dir = base / 'backend' / 'data'
out_dir.mkdir(parents=True, exist_ok=True)

insights = {
    'generated_at': pd.Timestamp.utcnow().isoformat(),
    'kpi_rows': len(agg),
    'advice_rows': len(advice),
    'top_bullish': sorted([a for a in advice if a['status']=='bullish'], key=lambda x: x['momentum'] or 0, reverse=True)[:50],
    'top_bearish': sorted([a for a in advice if a['status']=='bearish'], key=lambda x: x['momentum'] or 0)[:50],
}

with open(out_dir / 'mandi_insights.json', 'w', encoding='utf-8') as f:
    json.dump(insights, f, ensure_ascii=False, indent=2)

out_dir / 'mandi_insights.json'
